# Volumeter

In [68]:
import serial
import pandas as pd
import numpy as np
import time

In [141]:
global loadCellConstants

def readLoadCellCalibration():
    cal = np.genfromtxt('loadCellCalibration.csv', delimiter=',')
    return cal

def initMotionControl():
    motionControlPort = '/dev/ttyUSB1'
    return serial.Serial(port=motionControlPort, baudrate=115200)

def initSensors():
    global loadCellConstants
    loadCellConstants = readLoadCellCalibration()
    
    sensorPort = '/dev/ttyUSB0'
    return serial.Serial(port=sensorPort, baudrate=115200)

def closeSerial(serial):
    serial.close()

def sendMotionCommand(serial, cmd):
    serial.write('{0}\n'.format(cmd).encode('utf-8'))
    serial.flushInput()
    grbl_out = serial.readline()
    return grbl_out.decode()

def motionFinished(serial, wait=False):
    finished = False
    if(wait):
        while(not finished):
            time.sleep(0.5)
            grbl_out = sendMotionCommand(serial, '?')
            finished = 'Idle' in str(grbl_out)
        return True
    else:
        serial.write('?'.encode('utf-8'))
        serial.flushInput()
        grbl_out = sendMotionCommand(serial, '?')
        return 'Idle' in str(grbl_out)

def homingCycle(serial):
    cycle = ['?', 'G91', 'G38.2Z2500F2000', 'G1Z-20F1000',
             'G38.2X-560Z-560F300', 'G1X10Z10F300', 'G10L20P1X0Y0Z0',
            'G90', 'G1Z-1650F3000']
    for c in cycle:
        sendMotionCommand(serial, c)
    motionFinished(serial, wait=True)

def recordSensors(serial, numSamples):
    serial.flushInput()
    samplesRead = 0
    df = pd.DataFrame(columns=['time', 'loadCell', 'level'], dtype='float')
    while(samplesRead < numSamples):
        samplesRead += 1
        data = np.array(serial.readline().decode().rstrip().split(','))
        data = data.astype(np.float)
        df.loc[len(df)] = data
    df.set_index(['time'])
    return df

def meanWeight(serial, numSamples):
    data = recordSensors(serial, numSamples)
    return data['loadCell'].mean()

def calibrateLoadCell(serial, numWeights=1):
    print("aa")
    grams = []
    values = []
    print("Load Cell Calibration")
    print("Leave the specimen holder empty")
    input("Hit enter when the specimen holder is stationary")
    print("Wait for 5 seconds...")
    grams.append(0.0)
    values.append(meanWeight(serial, 80*5))
    while(numWeights > 0):
        numWeights -= 1
        while(True):
            try:
                calWeight = input("Mass of the calibration weight in kg: ").rstrip().replace(',', '.')
                grams.append(float(calWeight)*1000.0)
                break
            except:
                print("Insuitable mass value")

        print("Add the calibratrion weight to the specimen holder")
        input("Hit enter when the specimen holder is stationary")
        print("Wait for 5 seconds...")
        values.append(meanWeight(serial, 80*5))
    calibrationConstants = np.polyfit(values, grams, 1)
    print("Calibration: y = {:.2f}x+{:.2f}".format(calibrationConstants[0], calibrationConstants[1]))
    save = input("Save calibration (y/n)")
    if('y' in save.lower()):
        np.savetxt('loadCellCalibration.csv', calibrationConstants, delimiter=',')
        print('Calibration saved')
    else:
        print('Calibration cancelled')
        

In [143]:
mcSerial = initMotionControl()
finished = motionFinished(mcSerial, wait=True)
homingCycle(mcSerial)
mcSerial.close()

In [142]:
sensorSerial = initSensors()
#data = recordSensors(sensorSerial, 10)
#mw = meanWeight(sensorSerial, 100)
#calibrateLoadCell(sensorSerial, numWeights = 2)
closeSerial(sensorSerial)
print(loadCellConstants)

[-0.050 -6162.085]
